In [ ]:
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstRtspServer", "1.0")
from gi.repository import GLib, Gst, GstRtspServer
import pyds
import cv2
import numpy as np
import pytesseract

class MultiCameraLicensePlateRecognition:
    def __init__(self, cameras):
        self.running = False
        self.bitrate = 4000000
        self.pipeline = Gst.Pipeline()
        self.ocr = pytesseract

        streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
        if not streammux:
            sys.stderr.write("Unable to create NvStreamMux\n")
        self.pipeline.add(streammux)

        for i, camera in enumerate(cameras):
            print("Creating source for", camera.internal_id)
            source_bin = self.create_source_bin(i, camera.cam_address)

            if not source_bin:
                sys.stderr.write("Unable to create source bin\n")

            self.pipeline.add(source_bin)

            padname = "sink_%u" % i
            sinkpad = streammux.get_request_pad(padname)
            if not sinkpad:
                sys.stderr.write("Unable to create sink pad bin\n")

            srcpad = source_bin.get_static_pad("src")
            if not srcpad:
                sys.stderr.write("Unable to create src pad bin\n")
            srcpad.link(sinkpad)

        pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
        if not pgie:
            sys.stderr.write("Unable to create pgie\n")

        nvvidconv1 = Gst.ElementFactory.make("nvvideoconvert", "convertor1")
        if not nvvidconv1:
            sys.stderr.write("Unable to create nvvideoconvert\n")

        capsapp = Gst.ElementFactory.make("capsfilter", "capsapp")
        capsapp.set_property("caps", Gst.Caps.from_string("video/x-raw(memory:NVMM), format=RGBA"))

        appsink = Gst.ElementFactory.make("appsink", "appsink")

        if not appsink:
            sys.stderr.write("Unable to create appsink\n")

        appsink.set_property("emit-signals", True)
        appsink.set_property("sync", False)
        appsink.set_property("max-buffers", 35)  # Pseudo control of the framerate
        appsink.set_property("drop", True)
        appsink.connect("new-sample", self.on_new_sample, appsink, cameras)

        streammux.set_property('live-source', 1)
        streammux.set_property("width", 1280)
        streammux.set_property("height", 720)
        streammux.set_property("batch-size", len(cameras))
        streammux.set_property("batched-push-timeout", 20000)  # Ideally, set it to the latency of the fastest stream

        pgie.set_property("config-file-path", "configs/plate_detector_config.txt")
        pgie.set_property("batch-size", len(cameras))

        mem_type = int(pyds.NVBUF_MEM_CUDA_UNIFIED)
        streammux.set_property("nvbuf-memory-type", mem_type)
        nvvidconv1.set_property("nvbuf-memory-type", mem_type)

        self.pipeline.add(pgie)
        self.pipeline.add(nvvidconv1)
        self.pipeline.add(capsapp)
        self.pipeline.add(appsink)

        streammux.link(pgie)
        pgie.link(nvvidconv1)
        nvvidconv1.link(capsapp)
        capsapp.link(appsink)

        self.bus = self.pipeline.get_bus()
        self.bus.add_signal_watch()
        self.bus.connect("message", self.bus_call)

    def on_new_sample(self, sink, appsink, cameras):
        sample = sink.emit("pull-sample")
        gst_buffer = sample.get_buffer()
        if not gst_buffer:
            print("Unable to get GstBuffer")
            return Gst.FlowReturn.ERROR

        batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
        l_frame = batch_meta.frame_meta_list

        while l_frame is not None:
            try:
                frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
            except StopIteration:
                break

            l_obj = frame_meta.obj_meta_list
            obj_positions = []

            while l_obj is not None:
                try:
                    obj_meta = pyds.NvDsObjectMeta.cast(l_obj.data)
                except StopIteration:
                    break

                if obj_meta.class_id == 0:
                    n_frame = self.extract_object_region(obj_meta, frame_meta)
                    if n_frame is not None:
                        recognized_text = self.recognize_license_plate(n_frame)
                        obj_positions.append(recognized_text)

                try:
                    l_obj = l_obj.next
                except StopIteration:
                    break

            cameras[frame_meta.source_id].positionq.append(obj_positions)

            try:
                l_frame = l_frame.next
            except StopIteration:
                break

        return Gst.FlowReturn.OK

        # Extracting the region of the object from the frame
    def extract_object_region(self, obj_meta, frame_meta):
        rect_params = obj_meta.rect_params
        top = int(rect_params.top)
        left = int(rect_params.left)
        width = int(rect_params.width)
        height = int(rect_params.height)

        n_frame = pyds.get_nvds_buf_surface(hash(gst_buffer), frame_meta.batch_id)
        object_region = n_frame[top:top + height, left:left + width]

        return object_region

        # Recognizing text from the license plate region using OCR    
    def recognize_license_plate(self, image):
        recognized_text = pytesseract.image_to_string(image, config='--psm 6')
        return recognized_text

    def bus_call(self, bus, message):
        t = message.type
        if t == Gst.MessageType.EOS:
            sys.stdout.write("End-of-stream\n")
        elif t == Gst.MessageType.WARNING:
            err, debug = message.parse_warning()
            sys.stderr.write("Warning: %s: %s\n" % (err, debug))
        elif t == Gst.MessageType.ERROR:
            err, debug = message.parse_error()
            sys.stderr.write("Error: %s: %s\n" % (err, debug))
        return True

    def create_source_bin(self, index, uri):
        bin_name = f"source-bin-{index}"
        nbin = Gst.Bin.new(bin_name)

            # Creating a Source Bin for RTSP Stream
        if uri.startswith("rtsp://3.76.45.126:8554/stream1"):
            uri_decode_bin = Gst.ElementFactory.make("uridecodebin", f"uri-decode-bin-{index}")
            uri_decode_bin.set_property("uri", uri)
            uri_decode_bin.connect("pad-added", self.cb_newpad, nbin)
            uri_decode_bin.connect("child-added", self.decodebin_child_added, nbin)

            bin_pad = nbin.add_pad(Gst.GhostPad.new_no_target("src", Gst.PadDirection.SRC))
            if not bin_pad:
                sys.stderr.write("Failed to add ghost pad in source bin\n")
            return nbin

                # Creating a Source Bin for HLS Stream
        elif uri.startswith("https://test-streams.mux.dev/x36xhzz/x36xhzz.m3u8") or uri.startswith("http://129.125.136.20/camera/index.html"):
            if uri.endswith(".m3u8"):
                hls_source = Gst.ElementFactory.make("hlssource", f"hls-source-{index}")
                hls_source.set_property("location", uri)

                hls_demux = Gst.ElementFactory.make("hlsdemux", f"hls-demux-{index}")
                hls_demux.connect("pad-added", self.cb_newpad, nbin)

                bin_pad = nbin.add_pad(Gst.GhostPad.new_no_target("src", Gst.PadDirection.SRC))
                if not bin_pad:
                    sys.stderr.write("Failed to add ghost pad in source bin\n")
                return nbin

                # Creating a Source Bin for MJPEG Stream
            else:
                jpeg_source = Gst.ElementFactory.make("souphttpsrc", f"souphttpsrc-{index}")
                jpeg_source.set_property("location", uri)

                jpeg_demux = Gst.ElementFactory.make("jpegparse", f"jpegparse-{index}")

                bin_pad = nbin.add_pad(Gst.GhostPad.new_no_target("src", Gst.PadDirection.SRC))
                if not bin_pad:
                    sys.stderr.write("Failed to add ghost pad in source bin\n")
                return nbin
            

    def play(self):
        self.pipeline.set_state(Gst.State.PLAYING)

    def stop(self):
        self.pipeline.set_state(Gst.State.NULL)
